<a href="https://colab.research.google.com/github/hamzajb95/BoEDataToolsAnalyst/blob/main/Supervisory_Resource_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dependancies and and upload Data

In [202]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
# set a custom format for float values
pd.options.display.float_format = '{:.2f}'.format

In [203]:
#Upload Driver Manifest file 
sra_file = files.upload()

#Get the filename
for fn in sra_file.keys():
  print(fn)
  filename1 = fn



Saving BoE dataset.xlsx to BoE dataset.xlsx
BoE dataset.xlsx


# Set the dataframes

#### Basic Merge the dataset and a little cleaning

In [204]:
#Read the two sheets into dataframes
dfGeneral = pd.read_excel(filename1, sheet_name=0)
dfUW = pd.read_excel(filename1, sheet_name=1)

#Join the two datasets together to have all data for a firm for each year in one row
df = dfGeneral.merge(dfUW, on=['Firm','Year'], how ="inner")
df['Year'] = df['Year'].str.replace('YE','')
df = df.rename(columns={'Excess of assets over liabilities (£m) [= equity]':'equity'})
df.head(5)

,Firm,Year,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,Gross claims incurred (£m),"Gross BEL (inc. TPs as whole, pre-TMTP) (£m)","Net BEL (inc. TPs as a whole, pre-TMTP) (£m)",Pure net claims ratio,Net expense ratio,Net combined ratio,Pure gross claims ratio,Gross expense ratio,Gross combined ratio
0,Firm 1,2016,-13779.82,1085.36,2230.08,1.98,1409.48,71727.59,69696.56,2031.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Firm 2,2016,28.18,10.19,33.39,3.16,28.18,41.92,8.53,33.39,39.24,163.60,162.95,0.19,0.76,0.96,0.20,0.74,0.95
2,Firm 1,2017,0.00,0.00,4.82,481792000.00,0.00,4.82,0.00,4.82,0.05,7.67,7.67,11.40,56.81,68.22,11.40,56.81,68.22
3,Firm 4,2016,22344.20,16573.64,23862.76,1.39,29424.57,67404.35,50485.01,16919.34,17.13,-18.30,-8.08,0.88,0.20,1.08,0.70,0.14,0.85
4,Firm 5,2016,68.20,52.82,148.43,2.71,70.16,341.86,193.43,148.43,30.49,20.80,37.25,0.48,0.28,0.76,0.33,0.18,0.51


####checking which firms are inactive
If firm has 0 GWP and claims incurred for 3 consecutive years I will consider those firms closed

In [205]:
#Filter data for the last 3 years
filtered_df = df[(df['Year'].astype('int') >= 2018) & (df['Year'].astype('int') <= 2020)]

# Sum up all the values for each firm across last 3 years
sums = filtered_df.groupby('Firm').sum()

# Filter out firms where the total sum is 0
filtered_df = sums[(sums['GWP (£m)'] == 0) & (sums['Gross claims incurred (£m)'] == 0)]

#Reset index as Firm is set as the index.
filtered_df.reset_index(inplace=True)

filtered_df[['Firm','GWP (£m)','Gross claims incurred (£m)','Gross expense ratio']]




<ipython-input-205-19a8ef61f4ef>:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Firm,GWP (£m),Gross claims incurred (£m),Gross expense ratio
0,Firm 1,0.00,0.00,0.00
1,Firm 103,0.00,0.00,0.00
2,Firm 109,0.00,0.00,0.00
3,Firm 117,0.00,0.00,0.00
4,Firm 119,0.00,0.00,0.00
5,Firm 128,0.00,0.00,0.00
6,Firm 140,0.00,0.00,0.00
7,Firm 147,0.00,0.00,0.00
8,Firm 16,0.00,0.00,0.00
9,Firm 168,0.00,0.00,0.00


In [206]:
# create a boolean mask indicating which rows to keep
mask = ~df['Firm'].isin(filtered_df['Firm'])

# use the mask to filter out the rows to remove
df2 = df.loc[mask]

df2



,Firm,Year,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,Gross claims incurred (£m),"Gross BEL (inc. TPs as whole, pre-TMTP) (£m)","Net BEL (inc. TPs as a whole, pre-TMTP) (£m)",Pure net claims ratio,Net expense ratio,Net combined ratio,Pure gross claims ratio,Gross expense ratio,Gross combined ratio
1,Firm 2,2016,28.18,10.19,33.39,3.16,28.18,41.92,8.53,33.39,39.24,163.60,162.95,0.19,0.76,0.96,0.20,0.74,0.95
3,Firm 4,2016,22344.20,16573.64,23862.76,1.39,29424.57,67404.35,50485.01,16919.34,17.13,-18.30,-8.08,0.88,0.20,1.08,0.70,0.14,0.85
4,Firm 5,2016,68.20,52.82,148.43,2.71,70.16,341.86,193.43,148.43,30.49,20.80,37.25,0.48,0.28,0.76,0.33,0.18,0.51
5,Firm 6,2016,1826.80,2717.77,3481.00,1.23,1918.43,39780.70,36470.85,3309.85,411.24,716.58,590.11,0.83,0.25,1.08,1.03,0.24,1.26
6,Firm 7,2016,5855.17,3342.99,5120.18,1.48,6567.62,81043.29,74218.64,6824.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,Firm 316,2020,-0.04,225.33,420.42,1.80,-0.88,1277.73,827.55,450.18,70.88,131.20,8.59,0.64,1.11,1.75,0.64,0.37,1.02
1616,Firm 317,2020,2150.07,164.19,230.58,1.35,2150.07,15949.16,15714.73,234.43,191.05,452.48,378.49,0.68,0.37,1.05,0.66,0.31,0.98
1620,Firm 321,2020,24.63,167.55,295.16,1.70,46.13,6850.30,6555.13,295.16,3.47,0.49,0.49,0.54,0.28,0.82,0.54,0.28,0.82
1621,Firm 322,2020,2029.70,1395.28,2000.80,1.38,2179.24,5885.12,3749.42,2135.70,200.69,403.05,58.31,0.61,0.56,1.17,0.62,0.37,1.00


#### Cleaning Outliers

In [207]:
# setting Net combind ratio to 5 max
df2.loc[df2['Net combined ratio'] > 5, 'Net combined ratio'] = 5
df2.loc[df2['Net combined ratio'] < -0.5, 'Net combined ratio'] = -0.5
df2.loc[df2['SCR coverage ratio'] > 10, 'SCR coverage ratio'] = 10
df2.loc[df2['SCR coverage ratio'] < -0.5, 'SCR coverage ratio'] = -0.5


In [208]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 1 to 1623
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Firm                                          1495 non-null   object 
 1   Year                                          1495 non-null   object 
 2   NWP (£m)                                      1495 non-null   float64
 3   SCR (£m)                                      1495 non-null   float64
 4   EoF for SCR (£m)                              1495 non-null   float64
 5   SCR coverage ratio                            1495 non-null   float64
 6   GWP (£m)                                      1495 non-null   float64
 7   Total assets (£m)                             1495 non-null   float64
 8   Total liabilities (£m)                        1495 non-null   float64
 9   equity                                        1495 non-null   f

In [209]:
df2.describe()

,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,Gross claims incurred (£m),"Gross BEL (inc. TPs as whole, pre-TMTP) (£m)","Net BEL (inc. TPs as a whole, pre-TMTP) (£m)",Pure net claims ratio,Net expense ratio,Net combined ratio,Pure gross claims ratio,Gross expense ratio,Gross combined ratio
count,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00,1495.00
mean,818.23,366.66,570.35,2.37,999.27,7862.71,7294.55,568.15,149.45,319.94,207.22,632.82,967.43,0.66,1268.06,897.59,2165.64
std,3594.44,1452.25,2154.97,2.41,4018.16,31613.73,30218.55,2099.88,417.61,883.12,613.68,25334.71,37532.87,0.86,49432.39,34710.72,84142.55
min,-2305.85,0.00,-98.61,-0.50,-95.42,-217.43,-1521.33,-98.61,-98.00,-135.49,-104.84,-37350.64,-8766.06,-0.50,-25876.27,-8773.16,-34649.42
25%,0.00,2.37,8.36,1.26,0.00,20.16,3.53,8.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,10.85,19.47,42.59,1.67,20.38,151.44,87.78,44.27,5.61,14.72,7.10,0.28,0.24,0.81,0.34,0.23,0.76
75%,170.43,114.26,202.94,2.67,267.44,1179.62,836.76,197.67,111.05,231.78,141.70,0.63,0.42,1.01,0.66,0.35,0.98
max,48117.99,19600.26,28772.49,10.00,48117.99,332875.90,331981.94,24786.14,4487.91,10094.58,7539.12,978836.31,1451186.53,5.00,1911107.55,1342061.31,3253168.86


# Lets check what the market share for each company

so that we can determine the size of firms and the priority by ranking them based on market share

In [210]:
pd.options.display.float_format = '{:.2f}'.format
#First lets pivot the table to get the total NWP of each firm for 5 years.
firm_gwp_sum = pd.pivot_table(data=df2, index='Firm', values=['GWP (£m)'], aggfunc={'GWP (£m)':'sum'}, margins=False)

#Lets calculate the total Sum GWP for 5 years.
tot_GWP = firm_gwp_sum['GWP (£m)'].sum()
print(tot_GWP)


firm_gwp_sum['MarketShare'] = (firm_gwp_sum['GWP (£m)']/tot_GWP)*100
firm_gwp_sum.sort_values('MarketShare', ascending=False,inplace=True)
firm_gwp_sum.reset_index(inplace=True)
#Going to use the index as the Firm Size Rank
firm_gwp_sum.reset_index(inplace=True)
firm_gwp_sum['size_rank'] = firm_gwp_sum['index'] + 1
firm_gwp_sum.drop('index',axis=1,inplace=True)

firm_gwp_sum

1493902.2950998833


,Firm,GWP (£m),MarketShare,size_rank
0,Firm 210,198981.11,13.32,1
1,Firm 4,169285.78,11.33,2
2,Firm 26,81105.36,5.43,3
3,Firm 247,81073.40,5.43,4
4,Firm 34,79777.06,5.34,5
...,...,...,...,...
294,Firm 230,-0.09,-0.00,295
295,Firm 207,-0.21,-0.00,296
296,Firm 277,-0.96,-0.00,297
297,Firm 61,-9.10,-0.00,298


In [211]:
#Lets attach the associated rank to the original dataset for future use.
firm_rank = firm_gwp_sum[['Firm','size_rank']]
df2 = df2.merge(firm_rank, on=['Firm'], how ="left")


In [212]:
#What is the total number of firms that are being considered
print("There are total of ", firm_gwp_sum['Firm'].nunique()," firms")
pie_df = firm_gwp_sum
firm_gwp_sum.loc[firm_gwp_sum['size_rank'] > 25, 'Firm'] = 'Others'

# create pie chart
fig = go.Figure(data=[go.Pie(labels=pie_df['Firm'], values=pie_df['MarketShare'])])

# update layout
fig.update_layout(
    title="Market Share by Firm",
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="#7f7f7f"
    )
)

# show the plot
fig.show()


There are total of  299  firms


In [213]:
# filter out negative values
firm_size = firm_gwp_sum[firm_gwp_sum['MarketShare'] > 0]
print(firm_gwp_sum['Firm'].nunique())

#Create bar graph for Market share of each firm
fig = px.bar(firm_size, x='Firm', y='MarketShare', title='Market Share by Firm', hover_name='size_rank')
fig.show()

26


# Classify for Underwritting Risk

In [214]:
# create a new column for underwritting risk with a conditional statement based on whether net combined ratio is > 1
df2['uwRiskLvl'] = [0 if x < 1 else 1 for x in df2['Net combined ratio']]

def concat_years(series):
    return ', '.join([str(x) for x in series])

uwRisk = pd.pivot_table(data=df2, index='Firm', values=['uwRiskLvl','Year','Net combined ratio'], aggfunc={'uwRiskLvl':'sum','Year':concat_years,'Net combined ratio':'mean'}, margins=False)
uwRisk.sort_values('uwRiskLvl',ascending=False,inplace=True)
uwRisk.reset_index(inplace=True)

uwRisk = uwRisk.rename(columns={'Net combined ratio':'5yr NCR'})
uwRisk

,Firm,5yr NCR,Year,uwRiskLvl
0,Firm 316,1.76,"2017, 2018, 2019, 2016, 2020",5
1,Firm 114,1.18,"2016, 2017, 2018, 2019, 2020",5
2,Firm 227,1.10,"2016, 2017, 2018, 2019, 2020",5
3,Firm 241,1.13,"2016, 2017, 2018, 2019, 2020",5
4,Firm 165,1.13,"2016, 2017, 2018, 2019, 2020",5
...,...,...,...,...
294,Firm 17,0.92,"2016, 2017, 2020, 2018, 2019",0
295,Firm 167,0.00,"2016, 2020, 2017, 2018, 2019",0
296,Firm 282,0.00,"2016, 2017, 2018, 2019, 2020",0
297,Firm 283,0.90,"2016, 2017, 2018, 2019, 2020",0


In [215]:
# Create a bar graph for each uwRiskLvl using Plotly
fig = px.bar(uwRisk, x="Firm", y="5yr NCR", color="uwRiskLvl")

fig.add_shape(type='line',
              x0=0, x1=299, y0=1, y1=1,
              line=dict(color='red', width=3))

# Show the plot
fig.show()



In [216]:
df2 = df2.drop('uwRiskLvl',axis=1)
#Lets attach the associated uwRiskLvl and 5yrNCR to the original dataset for future use.
uwRisk = uwRisk[['Firm','5yr NCR','uwRiskLvl']]
df2 = df2.merge(uwRisk, on=['Firm'], how ="left")
df2

,Firm,Year,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,...,"Net BEL (inc. TPs as a whole, pre-TMTP) (£m)",Pure net claims ratio,Net expense ratio,Net combined ratio,Pure gross claims ratio,Gross expense ratio,Gross combined ratio,size_rank,5yr NCR,uwRiskLvl
0,Firm 2,2016,28.18,10.19,33.39,3.16,28.18,41.92,8.53,33.39,...,162.95,0.19,0.76,0.96,0.20,0.74,0.95,157,0.61,1
1,Firm 4,2016,22344.20,16573.64,23862.76,1.39,29424.57,67404.35,50485.01,16919.34,...,-8.08,0.88,0.20,1.08,0.70,0.14,0.85,2,1.40,4
2,Firm 5,2016,68.20,52.82,148.43,2.71,70.16,341.86,193.43,148.43,...,37.25,0.48,0.28,0.76,0.33,0.18,0.51,134,1.24,4
3,Firm 6,2016,1826.80,2717.77,3481.00,1.23,1918.43,39780.70,36470.85,3309.85,...,590.11,0.83,0.25,1.08,1.03,0.24,1.26,21,0.94,1
4,Firm 7,2016,5855.17,3342.99,5120.18,1.48,6567.62,81043.29,74218.64,6824.65,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,Firm 316,2020,-0.04,225.33,420.42,1.80,-0.88,1277.73,827.55,450.18,...,8.59,0.64,1.11,1.75,0.64,0.37,1.02,198,1.76,5
1491,Firm 317,2020,2150.07,164.19,230.58,1.35,2150.07,15949.16,15714.73,234.43,...,378.49,0.68,0.37,1.05,0.66,0.31,0.98,28,1.03,3
1492,Firm 321,2020,24.63,167.55,295.16,1.70,46.13,6850.30,6555.13,295.16,...,0.49,0.54,0.28,0.82,0.54,0.28,0.82,178,0.95,1
1493,Firm 322,2020,2029.70,1395.28,2000.80,1.38,2179.24,5885.12,3749.42,2135.70,...,58.31,0.61,0.56,1.17,0.62,0.37,1.00,27,1.46,5


In [217]:
uwRisk

,Firm,5yr NCR,uwRiskLvl
0,Firm 316,1.76,5
1,Firm 114,1.18,5
2,Firm 227,1.10,5
3,Firm 241,1.13,5
4,Firm 165,1.13,5
...,...,...,...
294,Firm 17,0.92,0
295,Firm 167,0.00,0
296,Firm 282,0.00,0
297,Firm 283,0.90,0


#Classify for Liquidity Risk


In [218]:
# create a new column for underwritting risk with a conditional statement based on whether net combined ratio is > 1
df2['liqRiskLvl'] = [0 if x >= 1 else 1 for x in df2['SCR coverage ratio']]

def concat_years(series):
    return ', '.join([str(x) for x in series])

liqRisk = pd.pivot_table(data=df2, index='Firm', values=['liqRiskLvl','Year','SCR coverage ratio'], aggfunc={'liqRiskLvl':'sum','Year':concat_years,'SCR coverage ratio':'mean'}, margins=False)


liqRisk = liqRisk.rename(columns={'SCR coverage ratio':'5yr SCRratio'})

# SCRratio = 0 means they dont have a SCR value.
liqRisk.loc[liqRisk['5yr SCRratio'] == 0, 'Risk Level'] = 'Safe'

liqRisk.sort_values('liqRiskLvl',ascending=False,inplace=True)
liqRisk.reset_index(inplace=True)


# create new column based on 5yr SCRratio values
liqRisk['Risk Level'] = liqRisk['5yr SCRratio'].apply(lambda x: 'High Risk' if x < 1 else ('Low Risk' if x < 1.4 else 'Safe'))


liqRisk

,Firm,5yr SCRratio,Year,liqRiskLvl,Risk Level
0,Firm 50,0.00,"2016, 2017, 2018, 2019, 2020",5,High Risk
1,Firm 64,0.23,"2016, 2017, 2018, 2019, 2020",5,High Risk
2,Firm 198,0.00,"2016, 2017, 2018, 2019, 2020",5,High Risk
3,Firm 251,0.00,"2016, 2017, 2018, 2019, 2020",5,High Risk
4,Firm 206,0.00,"2016, 2017, 2018, 2019, 2020",5,High Risk
...,...,...,...,...,...
294,Firm 228,2.22,"2016, 2017, 2018, 2019, 2020",0,Safe
295,Firm 229,1.33,"2016, 2017, 2018, 2019, 2020",0,Low Risk
296,Firm 23,3.37,"2016, 2017, 2018, 2019, 2020",0,Safe
297,Firm 230,2.04,"2016, 2017, 2018, 2019, 2020",0,Safe


In [219]:
# Create a bar graph for each uwRiskLvl using Plotly
fig = px.bar(liqRisk, x="Firm", y="5yr SCRratio", color="Risk Level")

fig.update_layout(legend_orientation='h')
fig.update_layout(legend_x=0.5, legend_y=1.2)
fig.update_layout(legend_font=dict(size=16))

fig.add_shape(type='line',
              x0=0, x1=299, y0=1, y1=1,
              line=dict(color='red', width=3))

# Show the plot
fig.show()


In [220]:
df2 = df2.drop('liqRiskLvl',axis=1)
#Lets attach the associated uwRiskLvl and 5yrNCR to the original dataset for future use.
liqRisk = liqRisk[['Firm','5yr SCRratio','Risk Level']]
df2 = df2.merge(liqRisk, on=['Firm'], how ="left")
df2

,Firm,Year,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,...,Net expense ratio,Net combined ratio,Pure gross claims ratio,Gross expense ratio,Gross combined ratio,size_rank,5yr NCR,uwRiskLvl,5yr SCRratio,Risk Level
0,Firm 2,2016,28.18,10.19,33.39,3.16,28.18,41.92,8.53,33.39,...,0.76,0.96,0.20,0.74,0.95,157,0.61,1,4.58,Safe
1,Firm 4,2016,22344.20,16573.64,23862.76,1.39,29424.57,67404.35,50485.01,16919.34,...,0.20,1.08,0.70,0.14,0.85,2,1.40,4,1.42,Safe
2,Firm 5,2016,68.20,52.82,148.43,2.71,70.16,341.86,193.43,148.43,...,0.28,0.76,0.33,0.18,0.51,134,1.24,4,2.98,Safe
3,Firm 6,2016,1826.80,2717.77,3481.00,1.23,1918.43,39780.70,36470.85,3309.85,...,0.25,1.08,1.03,0.24,1.26,21,0.94,1,1.32,Low Risk
4,Firm 7,2016,5855.17,3342.99,5120.18,1.48,6567.62,81043.29,74218.64,6824.65,...,0.00,0.00,0.00,0.00,0.00,8,0.00,0,1.49,Safe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,Firm 316,2020,-0.04,225.33,420.42,1.80,-0.88,1277.73,827.55,450.18,...,1.11,1.75,0.64,0.37,1.02,198,1.76,5,1.54,Safe
1491,Firm 317,2020,2150.07,164.19,230.58,1.35,2150.07,15949.16,15714.73,234.43,...,0.37,1.05,0.66,0.31,0.98,28,1.03,3,1.25,Low Risk
1492,Firm 321,2020,24.63,167.55,295.16,1.70,46.13,6850.30,6555.13,295.16,...,0.28,0.82,0.54,0.28,0.82,178,0.95,1,3.58,Safe
1493,Firm 322,2020,2029.70,1395.28,2000.80,1.38,2179.24,5885.12,3749.42,2135.70,...,0.56,1.17,0.62,0.37,1.00,27,1.46,5,1.37,Low Risk


# Determine which firm is changing most year on year.

In [221]:
df3 = df2
df3.sort_values(by=['Firm','Year'],ascending=True, inplace=True)

def calc_pct_change(df,col,newcol):
  # Calculate the percentage change in NWP for each firm between consecutive years
  df[newcol] = df.groupby('Firm')[col].pct_change()
  df.reset_index(inplace=True, drop=True)
  df[newcol].fillna(0, inplace=True)
  # replace all infinite values with 0
  df.replace([np.inf, -np.inf], 0, inplace=True)
  # setting pct_change range to max 10 and -10
  df.loc[df[newcol] > 10, newcol] = 10
  df.loc[df[newcol] < -10, newcol] = -10
  return df

df3 = calc_pct_change(df3,'NWP (£m) ','NWP_pct_change')
df3 = calc_pct_change(df3,'SCR (£m)','SCR_pct_change')
df3 = calc_pct_change(df3,'equity','equity_pct_change')
# df3 = calc_pct_change(df3,'GWP (£m)','GWP_pct_change')
df3 = calc_pct_change(df3,'Net BEL (inc. TPs as a whole, pre-TMTP) (£m)','NBEL_pct_change')
# df3[df3['Firm']=='Firm 99']
yoy = df3[['Firm','Year','NWP (£m) ','SCR (£m)','GWP (£m)','NWP_pct_change','SCR_pct_change','NBEL_pct_change','equity_pct_change','size_rank']] #'GWP_pct_change',
yoy

,Firm,Year,NWP (£m),SCR (£m),GWP (£m),NWP_pct_change,SCR_pct_change,NBEL_pct_change,equity_pct_change,size_rank
0,Firm 10,2016,1273.95,3628.70,1317.98,0.00,0.00,0.00,0.00,11
1,Firm 10,2017,10516.21,3127.30,10559.90,7.25,-0.14,0.34,-0.09,11
2,Firm 10,2018,10087.57,2386.91,10122.41,-0.04,-0.24,0.18,-0.15,11
3,Firm 10,2019,8921.88,2446.49,8960.73,-0.12,0.02,0.24,-0.02,11
4,Firm 10,2020,7893.06,2880.32,7923.37,-0.12,0.18,0.20,0.02,11
...,...,...,...,...,...,...,...,...,...,...
1490,Firm 99,2016,169.17,256.31,456.39,0.00,0.00,0.00,0.00,57
1491,Firm 99,2017,157.46,259.24,432.94,-0.07,0.01,-0.88,-0.08,57
1492,Firm 99,2018,165.58,244.99,453.16,0.05,-0.05,-0.45,0.06,57
1493,Firm 99,2019,175.48,245.56,473.87,0.06,0.00,-0.04,0.06,57


In [222]:
def yearOnYearChange(n,column_name):
  top_firms = yoy.nlargest(n, column_name)['Firm'].tolist()
  # bottom_firms = yoy.nsmallest(n, column_name)['Firm'].tolist()
  # firm_list = top_firms + bottom_firms

  # filter the DataFrame based on the list of firms
  scr_yoy = yoy[yoy['Firm'].isin(top_firms)]

  # Define the layout for the plot
  layout = {
      'title': column_name + ' by Year',
      'xaxis': {'title': 'Year'},
      'yaxis': {'title': column_name +' Percent Change'},
  }

  # Define the plot
  fig = px.line(scr_yoy, x='Year', y=column_name, color='Firm')

  # Set the layout
  fig.update_layout(layout)

  # Show the plot
  fig.show()

In [223]:
yearOnYearChange(10, 'SCR_pct_change')

In [224]:
yearOnYearChange(10, 'equity_pct_change')

In [225]:
std_dev = pd.pivot_table(data=df3, index='Firm', values=['NWP_pct_change','SCR_pct_change','NBEL_pct_change','equity_pct_change'],
                         aggfunc={'NWP_pct_change':'std', 'SCR_pct_change':'std','NBEL_pct_change':'std','equity_pct_change':'std'}, margins=False) #'GWP_pct_change' ,'GWP_pct_change':'std'

# std_dev['NWP_pct_change'] = std_dev['NWP_pct_change'].abs()
# std_dev['SCR_pct_change'] = std_dev['SCR_pct_change'].abs()
# std_dev['GWP_pct_change'] = std_dev['GWP_pct_change'].abs()


std_dev.reset_index(inplace=True)
std_dev.rename(columns={'NBEL_pct_change':'NBEL_stdev','NWP_pct_change':'NWP_stdev','SCR_pct_change':'SCR_stdev','equity_pct_change':'equity_stdev'},inplace=True) #'GWP_pct_change':'GWP_stdev'
# std_dev.sort_values('')


std_dev

,Firm,NBEL_stdev,NWP_stdev,SCR_stdev,equity_stdev
0,Firm 10,0.13,3.28,0.16,0.07
1,Firm 100,0.27,0.11,0.06,0.02
2,Firm 101,0.00,0.52,0.44,0.46
3,Firm 102,3.15,0.00,0.45,0.45
4,Firm 104,0.45,2.64,0.62,0.07
...,...,...,...,...,...
294,Firm 94,1.72,0.00,0.25,0.06
295,Firm 95,0.00,0.44,0.46,0.45
296,Firm 96,0.46,0.68,0.45,0.45
297,Firm 97,0.88,0.22,0.27,0.14


In [226]:
def rank(df, col, std_df, col2):
  #Keep only the column thats being ranked
  std_df = std_df[['Firm',col]]
  #sort descending for that column
  std_df.sort_values(col,ascending=False,inplace=True)
  #reset index and drop as it is not needed
  std_df.reset_index(inplace=True, drop=True)
  #Get new index+1 as your rank
  std_df.reset_index(inplace=True)
  std_df = std_df.rename(columns={'index':col2})
  std_df[col2] = std_df[col2]+1
  df = df.merge(std_df,on='Firm',how='left')
  return df


df2 = rank(df2,'NBEL_stdev',std_dev,'NBEL_dev_rank')
df2 = rank(df2,'equity_stdev',std_dev,'equity_dev_rank')
df2 = rank(df2,'NWP_stdev',std_dev,'NWP_dev_rank')
df2 = rank(df2,'SCR_stdev',std_dev,'SCR_dev_rank')


df2

<ipython-input-226-85e0dce01fb7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-226-85e0dce01fb7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-226-85e0dce01fb7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-226-85e0dce01fb7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

,Firm,Year,NWP (£m),SCR (£m),EoF for SCR (£m),SCR coverage ratio,GWP (£m),Total assets (£m),Total liabilities (£m),equity,...,equity_pct_change,NBEL_pct_change,NBEL_dev_rank,NBEL_stdev,equity_dev_rank,equity_stdev,NWP_dev_rank,NWP_stdev,SCR_dev_rank,SCR_stdev
0,Firm 10,2016,1273.95,3628.70,6402.43,1.70,1317.98,155205.16,148470.12,6735.04,...,0.00,0.00,172,0.13,221,0.07,24,3.28,152,0.16
1,Firm 10,2017,10516.21,3127.30,6214.46,1.91,10559.90,155343.13,149227.06,6116.07,...,-0.09,0.34,172,0.13,221,0.07,24,3.28,152,0.16
2,Firm 10,2018,10087.57,2386.91,4049.49,1.63,10122.41,136914.32,131719.42,5194.90,...,-0.15,0.18,172,0.13,221,0.07,24,3.28,152,0.16
3,Firm 10,2019,8921.88,2446.49,4233.17,1.67,8960.73,138456.98,133381.72,5075.26,...,-0.02,0.24,172,0.13,221,0.07,24,3.28,152,0.16
4,Firm 10,2020,7893.06,2880.32,4248.52,1.42,7923.37,142144.99,136976.89,5168.10,...,0.02,0.20,172,0.13,221,0.07,24,3.28,152,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,Firm 99,2016,169.17,256.31,401.20,1.51,456.39,857.04,448.00,409.03,...,0.00,0.00,102,0.37,231,0.06,218,0.05,232,0.07
1491,Firm 99,2017,157.46,259.24,365.35,1.36,432.94,961.97,586.92,375.05,...,-0.08,-0.88,102,0.37,231,0.06,218,0.05,232,0.07
1492,Firm 99,2018,165.58,244.99,396.68,1.56,453.16,1072.11,673.20,398.91,...,0.06,-0.45,102,0.37,231,0.06,218,0.05,232,0.07
1493,Firm 99,2019,175.48,245.56,421.21,1.65,473.87,1118.35,694.64,423.71,...,0.06,-0.04,102,0.37,231,0.06,218,0.05,232,0.07


In [227]:

df4 = df2[['Firm','Year','size_rank', '5yr NCR','uwRiskLvl', '5yr SCRratio', 'Risk Level','equity_dev_rank','SCR_dev_rank']] #'NWP_dev_rank', ,'NBEL_dev_rank'
df4 = df4[df4['Year']=='2020']
df4.drop('Year',axis=1,inplace=True)
df4 = df4.rename(columns={'Risk Level':'Liquidity_Risk_Level'})
df4

,Firm,size_rank,5yr NCR,uwRiskLvl,5yr SCRratio,Liquidity_Risk_Level,equity_dev_rank,SCR_dev_rank
4,Firm 10,11,1.05,2,1.67,Safe,221,152
9,Firm 100,152,0.91,2,1.14,Low Risk,281,247
14,Firm 101,12,0.00,0,1.10,Low Risk,50,76
19,Firm 102,262,0.87,0,2.00,Safe,61,62
24,Firm 104,17,0.20,0,6.43,Safe,217,18
...,...,...,...,...,...,...,...,...
1474,Firm 94,254,1.22,4,9.22,Safe,245,124
1479,Firm 95,241,0.00,0,6.00,Safe,55,50
1484,Firm 96,291,0.63,3,1.11,Low Risk,52,70
1489,Firm 97,196,1.29,3,2.44,Safe,154,121


In [228]:
file_name = "Firm_ranking.xlsx"

with pd.ExcelWriter(file_name) as writer:
  df4.to_excel(writer, sheet_name='Firms')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>